In [9]:
import pydicom as dicom # read the dicom files
import os # do directory operations
import pandas as pd # nice for data analysis

In [13]:
# /home/nuno/Downloads/datasets/kaggle-lung_cancer_2017
# /media/nuno/LaCie02/NUNO/datasets/stage1

data_dir = '/media/nuno/LaCie02/NUNO/datasets/stage1/'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('/home/nuno/Downloads/datasets/kaggle-lung_cancer_2017/stage1_labels.csv', index_col=0)


In [14]:
labels_df.head()

,cancer
id,
0015ceb851d7251b8f399e39779d1e7d,1
0030a160d58723ff36d73f41b170ec21,0
003f41c78e6acfa92430a057ac0b306e,0
006b96310a37b36cccb2ab48d10b49a3,1
008464bb8521d09a42985dd8add3d0d2,1


In [18]:
for patient in patients[:1]:
    label = labels_df.get_value(patient, 'cancer') # the label of cancer/no cancer for one patient
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    print(len(slices), slices[0].pixel_array.shape)
    #print(slices[0])
    

/home/nuno/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


195 (512, 512)


In [19]:
len(patients) # number of patients

1596

In [22]:
import matplotlib.pyplot as plt    

In [27]:
for patient in patients[:1]:
    label = labels_df.get_value(patient, 'cancer') # the label of cancer/no cancer for one patient
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    # sorting the slices in the proper order
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2])) 
    plt.imshow(slices[0].pixel_array)
    plt.show()

/home/nuno/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


KeyboardInterrupt: 

# This is to increase the size of the images

In [30]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 30, 20  # that's default image size for this interactive session
pylab.rcParams['font.family'] = 'sans-serif'
pylab.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
pylab.rcParams['font.serif'] = ['Bitstream Vera Sans']
pylab.rcParams["font.size"] = "10"

# 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

IMG_PX = 150

for patient in patients[:1]:
    label = labels_df.get_value(patient, 'cancer') # the label of cancer/no cancer for one patient
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    # sorting the slices in the proper order
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2])) 
    
    fig = plt.figure()
    for num, each_slice in enumerate(slices[:12]):
        y = fig.add_subplot(3,4, num+1)
        new_image = cv2.resize(np.array(each_slice.pixel_array), (IMG_PX, IMG_PX))
        
        y.imshow(new_image)
    
    plt.show()
        

/home/nuno/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
